In [2]:
import pandas as pd

In [3]:
# Importing Data
from data import Dream_report, df , stock_info
# Import data logic model revenue prediction
from main import overall_predicted_labels # 只有方向，要把ddate, stockid補上
from DreamReport import logit_direction

c:\Users\user1\Desktop\repo2\revenue-prediction\data.py:48: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Dream_report = pd.read_sql(qr_sub, conn)
c:\Users\user1\Desktop\repo2\revenue-prediction\data.py:72: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  stock_info = pd.read_sql(stock_qr_sub, conn)
c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\user1\anaconda3\envs\rev_pred_env\lib\site-packages\sklearn\linear_model\_logistic.py:460: Conv

In [4]:
#type(Dream_report['stockid'][0])
logit_direction['stockid'] = logit_direction['stockid'].astype(str)
logit_direction['predict_labels'] = logit_direction['predict_labels'].astype(float)
#print(logit_direction)

In [5]:
merged = pd.merge(logit_direction, Dream_report, on=['ddate', 'stockid'])
# Filter rows where 'predcited_labels' are different #兩模型預測相反的結果
result = merged[merged['predict_labels'] != merged['direction']]
#print(result)

In [6]:
print(result[result['ddate'] == 202108])

        ddate stockid  predict_labels  direction
45886  202108    1102             0.0        1.0
45887  202108    1103             0.0        1.0
45894  202108    1210             0.0        1.0
45896  202108    1215             0.0        1.0
45897  202108    1216             0.0        1.0
...       ...     ...             ...        ...
47615  202108    9945             0.0        1.0
47617  202108    9949             1.0        0.0
47621  202108    9958             0.0        1.0
47622  202108    9960             0.0        1.0
47623  202108    9962             1.0        0.0

[813 rows x 4 columns]


In [7]:
result['year'] = result['ddate'].astype(str).str[:4]

# Count the number of occurences for each year
year_counts = result['year'].value_counts()

print(year_counts)

year
2021    7832
2020    7410
2022    6859
2017    6714
2018    6709
2016    6706
2023    6429
2019    6410
Name: count, dtype: int64


C:\Users\user1\AppData\Local\Temp\ipykernel_9852\891335564.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['year'] = result['ddate'].astype(str).str[:4]


In [8]:
result['month'] = result['ddate'].astype(str).str[4:6]

# Count the number of occurences for each month
month_counts = result['month'].value_counts()

print(month_counts)

month
08    5511
04    5152
10    5130
09    5130
06    4996
07    4938
05    4923
11    4381
02    4156
12    4138
01    3788
03    2826
Name: count, dtype: int64


C:\Users\user1\AppData\Local\Temp\ipykernel_9852\3621829736.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['month'] = result['ddate'].astype(str).str[4:6]


In [9]:
march_rows = result[result['month'] == '03']

march_counts = march_rows['ddate'].value_counts()

print(march_counts)

ddate
202303    460
202003    426
202203    401
202103    396
201703    370
201603    287
201803    260
201903    226
Name: count, dtype: int64


In [10]:
aug_rows = result[result['month'] == '08']

aug_counts = aug_rows['ddate'].value_counts()

print(aug_counts)

ddate
202108    813
202308    748
201808    721
201908    675
202008    666
201608    639
202208    631
201708    618
Name: count, dtype: int64


In [11]:
date_counts = result['ddate'].value_counts()

print(date_counts)

ddate
202108    813
202110    763
202304    753
202011    751
202308    748
         ... 
202103    396
201703    370
201603    287
201803    260
201903    226
Name: count, Length: 94, dtype: int64


In [12]:
origin_counts = merged['ddate'].value_counts()
#print(origin_counts)

merged['Y'] = merged['ddate'].astype(str).str[:4]
origin_Ycounts = merged['Y'].value_counts()
print(origin_Ycounts)

merged['M'] = merged['ddate'].astype(str).str[4:6]
origin_Mcounts = merged['M'].value_counts()
print(origin_Mcounts) 

Y
2022    21105
2021    20816
2020    20446
2019    19990
2018    19566
2017    19080
2016    18364
2023    17799
Name: count, dtype: int64
M
10    13492
09    13466
08    13440
07    13406
06    13363
05    13336
04    13313
03    13305
02    13283
01    13276
12    11766
11    11720
Name: count, dtype: int64


In [13]:
# "每個月"兩模型預測相反的個股數 / 兩模型預測的總個股數
month_counts/origin_Mcounts*100

01    28.532691
02    31.288113
03    21.240135
04    38.699016
05    36.915117
06    37.386814
07    36.834253
08    41.004464
09    38.095945
10    38.022532
11    37.380546
12    35.169131
Name: count, dtype: float64

In [14]:
# "每年"兩模型預測相反的個股數 / 兩模型預測的總個股數
print(year_counts/origin_Ycounts*100)
max_date = (date_counts/origin_counts*100).idxmax()
min_date = (date_counts/origin_counts*100).idxmin()

# 預測相反比例最大、最小的日期
print(max_date)
print(min_date)

2016    36.517099
2017    35.188679
2018    34.289073
2019    32.066033
2020    36.241808
2021    37.624904
2022    32.499408
2023    36.120007
Name: count, dtype: float64
202108
201903


In [15]:
stock_info.rename(columns={'股票代號':'stockid', '指數彙編分類代號':'classi_code'}, inplace=True)
# 刪除重複個股資訊
stock_info_dedup = stock_info.drop_duplicates(subset='stockid')
# 將個股資訊加至result df
pred_merge_code = result.merge(stock_info_dedup, on='stockid', how='left')

In [16]:
print(pred_merge_code[pred_merge_code['ddate']==202108])

        ddate stockid  predict_labels  direction  year month 中文簡稱 上市上櫃  \
16202  202108    1102             0.0        1.0  2021    08   亞泥    1   
16203  202108    1103             0.0        1.0  2021    08   嘉泥    1   
16204  202108    1210             0.0        1.0  2021    08   大成    1   
16205  202108    1215             0.0        1.0  2021    08   卜蜂    1   
16206  202108    1216             0.0        1.0  2021    08   統一    1   
...       ...     ...             ...        ...   ...   ...  ...  ...   
17010  202108    9945             0.0        1.0  2021    08  潤泰新    1   
17011  202108    9949             1.0        0.0  2021    08   琉園    2   
17012  202108    9958             0.0        1.0  2021    08  世紀鋼    1   
17013  202108    9960             0.0        1.0  2021    08  邁達康    2   
17014  202108    9962             1.0        0.0  2021    08   有益    2   

      classi_code   指數彙編分類  
16202      C11010    傳產-水泥  
16203      C11010    傳產-水泥  
16204      C12010    傳產-

In [17]:
# 計算各種產業兩個模型判斷相反的次數，並降序排列
count_classi = pred_merge_code.groupby(['classi_code', '指數彙編分類']).size().reset_index(name='counts').sort_values(by='counts', ascending=False)
print(count_classi)

   classi_code          指數彙編分類  counts
7       C17020           傳產-生技    4347
67      C29020           傳產-其他    3932
14      C23010      電子上游-IC-設計    3101
59      C25010           傳產-營建    2719
4       C15010           傳產-電機    2039
..         ...             ...     ...
68      C29030          傳產-自行車     152
53      C23410       電子下游-手機製造     105
12      C22010           傳產-汽車     103
45      C23320       電子下游-電信服務      86
16      C23030  電子上游-IC-DRAM製造      49

[70 rows x 3 columns]


In [18]:
count_classi_y = pred_merge_code.groupby(['classi_code', '指數彙編分類', 'year']).size().reset_index(name='counts').sort_values(by=['year','counts'], ascending=False)
print(count_classi_y[count_classi_y['year']=='2023'])

print(count_classi_y[count_classi_y['year']=='2022'])

    classi_code          指數彙編分類  year  counts
63       C17020           傳產-生技  2023     506
543      C29020           傳產-其他  2023     485
119      C23010      電子上游-IC-設計  2023     388
479      C25010           傳產-營建  2023     295
39       C15010           傳產-電機  2023     238
..          ...             ...   ...     ...
431      C23410       電子下游-手機製造  2023      15
559      C30010         傳產-高爾夫球  2023      13
343      C23290       電子下游-數位相機  2023       9
367      C23320       電子下游-電信服務  2023       9
135      C23030  電子上游-IC-DRAM製造  2023       6

[70 rows x 4 columns]
    classi_code          指數彙編分類  year  counts
62       C17020           傳產-生技  2022     602
542      C29020           傳產-其他  2022     536
478      C25010           傳產-營建  2022     339
118      C23010      電子上游-IC-設計  2022     336
38       C15010           傳產-電機  2022     238
..          ...             ...   ...     ...
390      C23360        電子下游-掃描器  2022      14
430      C23410       電子下游-手機製造  2022      13
70       C1

In [19]:
# 將 merged(兩模型都有預測的個股)加上股票產業彙編碼
merged_merge_code = merged.merge(stock_info_dedup, on='stockid', how='left')
#print(merged_merge_code)

In [20]:
# 計算各產業數量(全，非僅包含預測相反者)
merge_count = merged_merge_code.groupby(['classi_code', '指數彙編分類']).size().reset_index(name='counts').sort_values(by=['counts'], ascending=False)
print(merge_count)

   classi_code          指數彙編分類  counts
7       C17020           傳產-生技   11502
67      C29020           傳產-其他   11068
14      C23010      電子上游-IC-設計    8069
59      C25010           傳產-營建    7488
4       C15010           傳產-電機    6479
..         ...             ...     ...
8       C18010         傳產-玻璃陶瓷     470
42      C23290       電子下游-數位相機     376
53      C23410       電子下游-手機製造     285
45      C23320       電子下游-電信服務     282
16      C23030  電子上游-IC-DRAM製造      94

[70 rows x 3 columns]


In [21]:
# 兩模型在各產業預測相反的比例 = 各產業不分時間預測相反總數量/兩模型都有預測的數量

# merge the df on 'classi_code'
ratio_merge = pd.merge(count_classi, merge_count, on=['classi_code', '指數彙編分類'], suffixes=('_c', '_m'))

# Calvulate the ratio and create new column
ratio_merge['ratio'] = ratio_merge['counts_c'] / ratio_merge['counts_m'] *100

# 依照占比降續排列
ratio_merge = ratio_merge.sort_values('ratio', ascending=False)

print(ratio_merge[['classi_code','指數彙編分類', 'ratio']])

   classi_code          指數彙編分類      ratio
69      C23030  電子上游-IC-DRAM製造  52.127660
42      C23050      電子上游-IC-製造  45.042840
48      C23020      電子上游-IC-代工  42.731278
50      C23300        電子下游-光碟片  42.401216
28      C23450           軟體-其他  42.161017
..         ...             ...        ...
21      C23430         軟體-系統整合  28.645674
41      C28010           金融-金控  27.963526
34      C23330       電子下游-工業電腦  27.016575
60      C23380      電子下游-筆記型電腦  22.813239
67      C22010           傳產-汽車  19.769674

[70 rows x 3 columns]


In [33]:
# 針對電子上游產業觀察兩模型預測1和0的數量
stock_df = stock_info_dedup.dropna()
stock_df[stock_df['指數彙編分類'].str.contains('電子上游')]

,stockid,中文簡稱,上市上櫃,classi_code,指數彙編分類
139,1569,濱川,2,C23070,電子上游-IC-通路
142,1582,信錦,1,C23110,電子上游-連接元件
151,1595,川寶,2,C23130,電子上游-PCB-材料設備
219,1815,富喬,2,C23130,電子上游-PCB-材料設備
295,2254,巨鎧精密-創,7,C23100,電子上游-LED及光元件
...,...,...,...,...,...
5121,6594,展匯科,2,C23010,電子上游-IC-設計
6170,3383,新世紀,1,C23100,電子上游-LED及光元件
6788,6251,定穎,1,C23120,電子上游-PCB-製造
6812,6289,華上,1,C23100,電子上游-LED及光元件
